In [1]:
# Pradėta 2024-11-21_ObjectProperties.ipynb
# Tęsinys. Čia 2024-12-11...

import pandas as pd
import time
import sqlite3
import random

from bs4 import BeautifulSoup

# Selenium
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver

from my_lib.properties_of_7 import properties_of
from my_lib.my_db_functions import df_to_sqlite

In [ ]:
# Nustatymai, parametrai:

item_type = 'mobilieji-telefonai'
# 'dronai', 'televizoriai', 'saldytuvai', 'televizoriai'
item_params = 'f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&'

table_version = '_9'
item_producer = 'samsung'

common_prop = {'producer':'samsung','5g':True} # Common properties

url = f'https://www.varle.lt/{item_type}/?{item_params}'
print(url)

https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&


In [ ]:
varlė=properties_of('varlė',engine='pandas')

In [4]:
# --- Trinu. Nereikalingas. Viską išsaugosiu per 'properties_of'.

# Šis fragmentas automatiškai išvalo 'items_df' sukauptus duomenis,
# Pasirengta naujų duomenų įrašymui.

# items_df = pd.DataFrame(columns=('id','price','url','price_in_cents','producer','5g'))
# # čia bus šios programos rezultatas

# items_df

In [ ]:
# SCRAP

# Atidaro Chrome naršyklę,
# reikia patvirtinti "Leisti visus slapukus"!
# Šis fragmentas automatiškai panaikina „draiver“.

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)

driver.get(url)
# time.sleep(3)
source = driver.page_source
print(f'First page: {url}')

bs = BeautifulSoup(source,'html.parser')
pages_numbers = bs.find('div',{'class':'PAGINATION'}).find_all('li',{'class':'number'})
total_pages = int(pages_numbers[-1].text.strip())
print(f'Total pages: {total_pages}')

First page: https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&
Total pages: 6


In [ ]:
###############################################################################

In [ ]:
# SCRAP

# Toliau analizuojamas prekių sąrašo html iš bs=BeautifulSoup().

items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
for item_block in items_blocks:

    item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
    item_url = 'https://www.varle.lt'+ item_href
    item_id = item_href.split('-')[-1].split('.')[0]

    price_container = item_block.find('div',{'class':'price-container'})
    price_value = price_container.find('span',{'class':'price-value'}).getText()
    item_price = ''.join(price_value.split())

    # items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}
    

print(f'DF items: {len(items_df)}.')

DF items: 38.


In [1]:
# item_block
items_df

NameError: name 'items_df' is not defined

In [7]:
def str_str(m):
    if len(m) == 0:
        return ""
    elif len(m) == 1:
        return m[0].lower()
    else:
        return f"{m[0]}_{m[1]}".lower()

li_params = ['gamintojas','ekrano_įstrižainė','vidinė_atmintis','atmintis_(ram)','2_sim']
for listitem in item_block.find('div',{'class':'spec-shortcuts'}).find_all('li'):
    print(listitem)
    li_contents=listitem.contents[0].split()
    print(str_str(li_contents))
    print(listitem.find('span').text)
    print()

<li>Gamintojas<a href="/mobilieji-telefonai/?f.s-gamintojas=Samsung"><span>Samsung</span></a></li>
gamintojas
Samsung

<li>Ekrano įstrižainė (coliais)<a href="/mobilieji-telefonai/?f.r.float-ekrano_istrizaine=6.4%3Ainf"><span>6.8</span></a></li>
ekrano_įstrižainė
6.8

<li>Spalva<a href="/mobilieji-telefonai/?f.s-spalva=Juoda"><span>Juoda</span></a></li>
spalva
Juoda

<li>Vidinė atmintis (GB)<a href="/mobilieji-telefonai/?f.s-vidine_atmintis=256" rel="nofollow"><span>256</span></a></li>
vidinė_atmintis
256

<li>Atmintis (RAM) (GB)<a href="/mobilieji-telefonai/?f.s-atmintis_ram=12" rel="nofollow"><span>12</span></a></li>
atmintis_(ram)
12

<li>2 SIM kortelės<a href="/mobilieji-telefonai/?f.s-2_sim_korteles=Taip"><span>Taip</span></a></li>
2_sim
Taip



In [8]:
# SCRAP

# Perskaitomi kiti prekių sąrašo puslapiai,
# po to įrašoma į db

import random

# for page_number in range(2,total_pages+1):
for page_number in range(2,4):

    url = f'https://www.varle.lt/{item_type}/?{item_params}p={page_number}'
    driver.get(url)
    time.sleep(random.randint(25,75))
    source = driver.page_source
    print(f'Prskaityta {url}')

    bs = BeautifulSoup(source,'html.parser')

    items_blocks = bs.find_all('div',{'class':'GRID_ITEM'})
    for item_block in items_blocks:
        item_href = item_block.find('div',{'class':'product-title'}).find('a')['href']
        item_url = 'https://www.varle.lt'+item_href
        item_id = item_href.split('-')[-1].split('.')[0]

        price_container = item_block.find('div',{'class':'price-container'})
        price_value = price_container.find('span',{'class':'price-value'}).getText()
        item_price = ''.join(price_value.split())

        # items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}
        items_df.loc[len(items_df)] = {'id':int(item_id),'price':item_price,'url':item_url}

    items_total = len(items_df)
    items_df.drop_duplicates(subset='id',keep='last',inplace=True)
    print(f'Get {items_total} items. Unique {len(items_df)} items.')

# driver.close() # Uždaro Chrome puslapį.

Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&p=2
Get 76 items. Unique 76 items.
Prskaityta https://www.varle.lt/mobilieji-telefonai/?f.s-5g_rysys=Yra&f.s-gamintojas=Samsung&sort=-price&p=3
Get 114 items. Unique 114 items.


In [48]:
item_block

<div class="GRID_ITEM">
<div class="img-container">
<a href="/mobilieji-telefonai/telefonas-galaxy-s21-5g128gb-rozinis-sm-g991b-samsung--15154304.html">
<img alt="Nuotrauka Nr.1" class="product-img primary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/281/sam/samsung-galaxy-s21-5g-128gb-phantom-rozine.jpg" title="Telefonas GALAXY S21 5G/128GB Rožinis SM-G991B SAMSUNG" width="235"/>
<img alt="Nuotrauka Nr.2" class="product-img secondary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/622/ism/ismanusis-telefonas-samsung-galaxy-s21-5g_IuULkpN.jpg" title="Telefonas GALAXY S21 5G/128GB Rožinis SM-G991B SAMSUNG" width="235"/>
</a>
<div class="grid-labels">
</div>
<span class="product-like" id="15154304"></span>
</div>
<div class="product-info">
<ul class="sale-guarantee">
</ul>
<div class="product-title">
<a href="/mobilieji-telefonai/telefonas-galaxy-s21-5g128gb-rozinis-sm-g991b-samsung--15154304.html" t

In [50]:
# Pridedamas kainos stulpelis centais (int)

import re

def price_in_cents(price:str) -> int:
    # Pašalina visus simbolius, išskyrus skaičius, kablelį ir tašką:
    cleaned = re.sub(r'[^\d.,]', '', price).replace(',', '.').replace(' ', '')
    return int(float(cleaned)*100)

items_df['price_in_cents'] = items_df['price'].apply(price_in_cents)

items_df['producer'] = 'samsung'
items_df['5g'] = True

In [ ]:
# Užrašai:
# dfpv5['svoris:'] = dfpv5['svoris:'].apply(lambda x: float(x)*1000 if '.' in str(x) else x)

In [56]:
items_df

,id,price,url,price_in_cents,producer,5g
0,37056924,2379€,https://www.varle.lt/mobilieji-telefonai/samsu...,237900,samsung,True
1,37051688,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung,True
2,37054437,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung,True
3,22867446,2109.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,210998,samsung,True
4,22970524,1999.99€,https://www.varle.lt/mobilieji-telefonai/isman...,199999,samsung,True
...,...,...,...,...,...,...
109,32852030,449€,https://www.varle.lt/mobilieji-telefonai/samsu...,44900,samsung,True
110,32852085,449€,https://www.varle.lt/mobilieji-telefonai/samsu...,44900,samsung,True
111,24542650,445.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,44599,samsung,True
112,23875430,443.45€,https://www.varle.lt/mobilieji-telefonai/telef...,44345,samsung,True


In [54]:
f'{item_type}{table_version}'

'mobilieji-telefonai_9'

In [55]:
print('Save DF to SQLite3. Plati lentelė, ne properties_of()')

df_to_sqlite(items_df,"./my_scrap_working_data.db",table_name=f'{item_type}{table_version}')

Save DF to SQLite3. Plati lentelė, ne properties_of()
Lentelė 'mobilieji-telefonai_9' sėkmingai įrašyta į DB.


In [57]:
print('Iš DF kuriamas properties_of() ir įrašomas į DB.')

mobilieji_telefonai = properties_of( "mobilieji_telefonai",
    engine="sqlite_file", db_path="./my_scrap_prop_of.db")

mobilieji_telefonai.import_from_df(items_df,id='id')

Iš DF kuriamas properties_of() ir įrašomas į DB.


In [61]:
items_df

,id,price,url,price_in_cents,producer,5g
0,37056924,2379€,https://www.varle.lt/mobilieji-telefonai/samsu...,237900,samsung,True
1,37051688,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung,True
2,37054437,2139€,https://www.varle.lt/mobilieji-telefonai/samsu...,213900,samsung,True
3,22867446,2109.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,210998,samsung,True
4,22970524,1999.99€,https://www.varle.lt/mobilieji-telefonai/isman...,199999,samsung,True
...,...,...,...,...,...,...
109,32852030,449€,https://www.varle.lt/mobilieji-telefonai/samsu...,44900,samsung,True
110,32852085,449€,https://www.varle.lt/mobilieji-telefonai/samsu...,44900,samsung,True
111,24542650,445.99€,https://www.varle.lt/mobilieji-telefonai/samsu...,44599,samsung,True
112,23875430,443.45€,https://www.varle.lt/mobilieji-telefonai/telef...,44345,samsung,True


In [62]:
# Čia galima pradėti dirbti su DF, nuskaitytu iš DB.

mobilieji_telefonai = properties_of( "mobilieji_telefonai",
    engine="sqlite_file", db_path="./my_scrap_prop_of.db")

In [63]:
print('Atkuriamas DF iš properties_of()')
print(mobilieji_telefonai.export_to_df(['5g','price']))

Atkuriamas DF iš properties_of()
property_id object_id 5g     price
0            14511169  1   739.99€
1            14522358  1   725.99€
2            15087466  1   439.99€
3            15124168  1   512.75€
4            15124169  1   689.99€
..                ... ..       ...
216          37974184  1   249.99€
217          38371553  1   235.99€
218          39080345  1   355.99€
219          39090142  1   499.99€
220          40650587  1  1149.99€

[221 rows x 3 columns]


In [26]:
items_df=mobilieji_telefonai.export_to_df()

In [27]:
items_df

property_id,object_id,5g,create,price,price_in_cents,producer,type,update,url
0,14511169,1,1733334394,729.99€,72999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
1,14522358,1,1733334394,709.99€,70999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/isman...
2,15087466,1,1733334394,439.99€,43999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
3,15124168,1,1733334394,511.38€,51138,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
4,15124169,1,1733334394,689.99€,68999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
...,...,...,...,...,...,...,...,...,...
211,37974184,1,1733334394,249.99€,24999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
212,38371553,1,1733334394,235.99€,23599,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
213,39080345,1,1733334394,355.99€,35599,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
214,39090142,1,1733334394,280.30€,28030,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/telsa...


In [28]:
import time

items_df['type'] = 'mobilieji-telefonai'
items_df['create'] = round(time.time())
items_df['update'] = round(time.time())

In [32]:
print('Iš DF kuriamas properties_of() ir įrašomas į DB.')

Iš DF kuriamas properties_of() ir įrašomas į DB.


In [33]:

varle_items_prop = properties_of( "varle_items",
    engine="sqlite_file", db_path="./my_scrap_prop_of.db")


In [43]:

varle_items_prop.import_from_df(items_df,id='object_id')


In [44]:

varle_items_prop.export_to_df().head(7)

property_id,object_id,5g,create,price,price_in_cents,producer,type,update,url
0,14511169,1,1733334394,729.99€,72999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
1,14522358,1,1733334394,709.99€,70999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/isman...
2,15087466,1,1733334394,439.99€,43999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
3,15124168,1,1733334394,511.38€,51138,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
4,15124169,1,1733334394,689.99€,68999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/samsu...
5,15154304,1,1733334394,439.99€,43999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/telek...
6,15154309,1,1733334394,629.99€,62999,samsung,mobilieji-telefonai,1733334394,https://www.varle.lt/mobilieji-telefonai/mobil...
